In [ ]:
import ee
import geopandas as gpd

ee.Authenticate()
ee.Initialize(project="ee-tiagogoncalves")

In [6]:
from pathlib import Path

input_dir = Path("../data/raw")
output_dir = Path("../data/feature")
output_dir.mkdir(exist_ok=True)

gpkg_files = list(input_dir.glob("*.gpkg"))

In [3]:
def get_download_url(xmin, ymin, xmax, ymax, year):
    geometry = ee.Geometry.Rectangle([[xmin, ymin], [xmax, ymax]], proj='EPSG:3857')

    s_date = ee.Date.fromYMD(year, 1, 1)
    e_date = s_date.advance(1, 'year')
                
    collection = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
    
    image = collection.filterBounds(geometry).filterDate(s_date, e_date).first()
    
    return image.getDownloadURL({
        'scale': 10,
        'region': geometry,
        'format': 'GEO_TIFF'
    })


def download_from_url(filename, url):
    import requests
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(output_dir / f"{filename}.tif", 'wb') as f:
            f.write(response.content)
        print(f"    ✓ Salvo: {filename}")
    else:
        print(f"    ❌ Erro ao baixar: Status {response.status_code}")

In [ ]:
from time import sleep

erros = []

for gpkg_file in gpkg_files:
    gdf = gpd.read_file(gpkg_file)

    xmin, ymin, xmax, ymax = gdf.total_bounds

    for attempt in range(3):
        try:
            url = get_download_url(xmin, ymin, xmax, ymax, 2024)

            download_from_url(gpkg_file.stem, url)

            break
        except Exception as e:
            if attempt == 2:
                erros.append((gpkg_file.stem, e))

            print(f"    ❌ Erro ao baixar a imagem: {e}. Tentando novamente...")
            sleep(2**attempt)

print("✅ Processamento concluído!")

    ✓ Salvo: 1-mvp
    ✓ Salvo: 10-mvp
    ✓ Salvo: 1000-mvp
    ✓ Salvo: 1001-mvp
    ✓ Salvo: 1002-mvp
    ✓ Salvo: 1003-mvp
    ✓ Salvo: 1004-mvp
    ✓ Salvo: 1005-mvp
    ✓ Salvo: 1007-mvp
    ✓ Salvo: 1008-mvp
    ✓ Salvo: 1009-mvp
    ✓ Salvo: 101-mvp
    ✓ Salvo: 1010-mvp
    ✓ Salvo: 1011-mvp
    ✓ Salvo: 1012-mvp
    ✓ Salvo: 1013-mvp
    ✓ Salvo: 1014-mvp
    ✓ Salvo: 1015-mvp
    ✓ Salvo: 1016-mvp
    ✓ Salvo: 1019-mvp
    ✓ Salvo: 1020-mvp
    ✓ Salvo: 1021-mvp
    ✓ Salvo: 1022-mvp
    ✓ Salvo: 1023-mvp
    ✓ Salvo: 1025-mvp
    ✓ Salvo: 1026-mvp
    ✓ Salvo: 1027-mvp
    ✓ Salvo: 1029-mvp
    ✓ Salvo: 103-mvp
    ✓ Salvo: 1030-mvp
    ✓ Salvo: 1031-mvp
    ✓ Salvo: 1032-mvp
    ✓ Salvo: 1033-mvp
    ✓ Salvo: 1034-mvp
    ✓ Salvo: 1035-mvp
    ✓ Salvo: 1036-mvp
    ✓ Salvo: 1037-mvp
    ✓ Salvo: 1038-mvp
    ✓ Salvo: 1039-mvp
    ✓ Salvo: 104-mvp
    ✓ Salvo: 1040-mvp
    ✓ Salvo: 1042-mvp
    ✓ Salvo: 1043-mvp
    ✓ Salvo: 1045-mvp
    ✓ Salvo: 1046-mvp
    ✓ Salvo: 1047-